# Monad transformers

## Outline

* Incentive for the Monad transformers

* The MonadIO type class

* Monad Transformers

In this lesson, we will learn about Monad transformers and how you can use them.

## Incentive for Monad transformers

When we learned about the State monad we showed how you can implement the Tic-Tac-Toe with using the State monad. 

A curious student might ask how to implement this game with the State monads by geting the input from the user instead of the random funtion.

Or in other words, is it possible to perform IO actions within a user defined State monad? The answer is yes.

With the knowledge that we have so far we can create a custom State monad that we will call `StateIO`.

The idea is that the result of the `runState` function will have to be wrapped in IO.

Below is the code that implements our costum `StateIO` state monad that returns the result in IO context.

In [ ]:
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE FlexibleInstances #-} 

import Control.Monad.Trans
import Control.Monad (ap, liftM)
import Control.Monad.State.Class (MonadState(get, put))

newtype StateIO s a = StateIO {runStateIO :: s -> IO (a, s)}

instance Functor (StateIO s) where
    fmap = liftM

instance Applicative (StateIO s) where
    pure = return
    (<*>) = ap

instance Monad (StateIO s) where
    return a = StateIO $ \s -> return (a, s)
    n >>= k = StateIO $ \s -> do (a, s') <- runStateIO n s
                                 runStateIO (k a) s'

instance MonadState s (StateIO s) where
    get = StateIO $ \st -> return (st,st)
    put st = StateIO $ const $ return ((),st)

We see that the `StateIO` type is defined similar as the `State` type just that it returns `IO (a,s)` instead of `(a,s)`.

But if we try now and create a state IO monad and try to perform some IO action in it, the code will not work.

In [ ]:
myStateIO :: StateIO [Int] ()
myStateIO = do
    st <- get
    print $ head st -- causes compilation error
    return ()

main :: IO ()
main = do
    (_, st) <- runStateIO myStateIO [1..3]
    putStrLn "Finished."

main

The reason for this is how the bind operator for the `StateIO` is defined. 
```haskell
(>>=) :: StateIO s a -> (a -> StateIO s b) -> StateIO s b
```

It can not take in a non `StateIO` variable variable or a function that would not return a `StateIO` variable. 

And the `print` function returns only `IO ()` and not `StateIO () a`, so the bind operator can not be applied to it.

The solution to this is defining a function that takes an IO action and lifts it to a `StateIO` monad.

In [ ]:
liftIOtoStateIO :: IO a -> StateIO s a
liftIOtoStateIO io = StateIO $ \st -> do x <- io
                                         return (x, st)

Now we can write again our code and it will compile correctly.

In [ ]:
myStateIO :: StateIO [Int] ()
myStateIO = do
    st <- get
    liftIOtoStateIO $ print $ head st
    return ()

main :: IO ()
main = do
    (_, st) <- runStateIO myStateIO [1..3]
    putStrLn "Finished."

main

We created now a State monad that works in IO context and found a way how to perform IO actions inside of such a monad. 

Of course you could do the same and create a State monad that returns the result in a Maybe context or as a list of tuples.

Such transformations make code more practical. For this reason Haskell provides build in solutions for them.

## The MonadIO type class

Because lifting a type wraped inside IO to another monad is very usefull, Haskell handles this with the `MonadIO` type class.

It defines the `liftIO` function that generalizes the type signature of our `liftIOtoStateIO` function.
```haskell
class Monad m => MonadIO m where
    liftIO :: IO a -> m a 
```

The `MonadIO` type class is defined in the **Control.Monad.IO.Class** module which is part of the **base** package.

So insted of defining a custom function for lifting we can import this module and create an instance of `MonadIO` for the `StateIO s` type.

In [ ]:
import Control.Monad.IO.Class

instance MonadIO (StateIO s) where
    liftIO io = StateIO $ \st -> do x <- io
                                    return (x, st)

myStateIO :: StateIO [Int] ()
myStateIO = do
    st <- get
    liftIO $ print $ head st
    return ()

## Monad Transformers

Next we ask ourselfs how to generalize wrapping the result of the State monad function into a monadic context.

When trying to wrap an existing monad into another monad we call the function that does this a **monad transformer**.

There are many transformers already defined in various modules. For the State monad we can use the `StateT` monad transformer. 

It is defined similar as the State monad type and is provided in the **Control.Monad.Trans.State.Lazy** module that comes with the **transformers** package.

```haskell
newtype StateT s m a = StateT { runStateT :: s -> m (a,s) }

instance Monad m => Monad (StateT s m) where
    return a = StateT $ \ s -> return (a, s)
    m >>= k  = StateT $ \ s -> do
        ~(a, s') <- runStateT m s
        runStateT (k a) s'
```

The `~` operator is used for ...

When defining an Functor and Applicative instance for this type we also need to add the Monad type constraint.
```haskell
instance Monad m => Functor (StateT s m) where
    fmap = liftM 

instance Monad m => Applicative (StateT s m) where
    pure = return
    (<*>) = ap
```

Now we can re-write the previous code example with the `StateIO` monad in this way:

In [ ]:
import Control.Monad.Trans.State.Lazy



## Recap

In this lesson we've discussed:

- the motivation for introducing Monad transformers

- The MonadIO type class

- Monad Transformers